In [13]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

In [14]:
#Images
# image = mpimg.imread('test_images/solidYellowLeft.jpg')
video = cv2.VideoCapture('test_videos/solidYellowLeft.mp4')

#Canny image parameters
kernel_size = 5
low_threshold = 70
high_threshold = 210

#Line parameters
rho = 1
theta = np.pi/180
threshold = 5
min_line_length = 5
max_line_gap = 1

In [15]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    return edges

def region_of_interest(edges):
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    imshape = image.shape
    vertices = np.array([[(40, imshape[0]), (430, 330), (530, 330), (890, imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)
    return masked_edges

def draw_lines(lines, line_image):
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1,y1), (x2,y2),(255,0,0),10)
    return line_image

def average_lines(image, lines):
    left_fit = []
    right_fit = []
    left_intercept = []
    right_intercept = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1)/(x2 - x1)
            if slope < 0:
                left_intercept.append((y1-x1*slope, y2-x2*slope))
                left_fit.append(slope)
            else:
                right_fit.append(slope)
                right_intercept.append((y1-x1*slope, y2-x2*slope))
    
    right_fit_average = np.mean(right_fit)
    left_fit_average = np.mean(left_fit)
    right_int_average = np.mean(right_intercept)
    left_int_average = np.mean(left_intercept)
    right_line = make_coordinate(image, right_fit_average, right_int_average)
    left_line = make_coordinate(image, left_fit_average, left_int_average)
    return np.array([left_line, right_line])

def make_coordinate(image, slope, intercept):
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

while(video.isOpened()):
    _, image = video.read()
    if image is None:
        break
    line_image = np.copy(image)*0
    canny_image = canny(image)
    region = region_of_interest(canny_image)
    lines = cv2.HoughLinesP(region, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    averaged_lines = average_lines(image, lines)
    line_image = draw_lines(averaged_lines, line_image)

    color_edges = np.dstack((canny_image, canny_image, canny_image))
    combo = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    cv2.imshow('Combined', combo)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video.release()
cv2.destroyAllWindows()
